<a href="https://colab.research.google.com/github/NataliaCondori/IMT344-Project/blob/main/MediaPipe_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 20.5 MB/s eta 0:00:00


In [ ]:
# Importing necessary libraries
import os  # Provides a way of using operating system dependent functionality
import numpy as np  # NumPy is the fundamental package for array computing with Python
import matplotlib.pyplot as plt  # Matplotlib is a plotting library for Python and NumPy
import cv2  # OpenCV (cv2) is used for computer vision tasks
from google.colab.patches import cv2_imshow  # Google Colab function for image display
import mediapipe as mp  # MediaPipe is a cross-platform framework for building multimodal applied machine learning pipelines

In [3]:
# Import necessary libraries
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_principal = r"/content/drive/MyDrive/VISION_PROYECTO/Dataset Proyecto "

# List files in the main folder
!ls "{folder_principal}"

'01 Gracias'	     '05 Papa'	   '09 Abuelo'	 '13 Esposo'	'17 Puente'
'02 Muchas_gracias'  '06 Mama'	   '10 Tio'	 '14 Primo'	'18 Calle'
'03 Bienvenidos'     '07 Hermano'  '11 Sobrino'  '15 Casa'	'19 Avenida'
'04 Lo_espero'	     '08 Hijo'	   '12 Esposa'	 '16 Hospital'	'20 Plaza'


In [ ]:
# Get subdirectories in the main folder
folder_palabras = [os.path.join(folder_principal, d) for d in os.listdir(folder_principal) if os.path.isdir(os.path.join(folder_principal, d))]
folder_palabras.sort()

In [ ]:
# Get folder names (without full paths) in the main folder
folder_names = [os.path.basename(d) for d in os.listdir(folder_principal) if os.path.isdir(os.path.join(folder_principal, d))]
folder_names.sort()

In [ ]:
video_files=[]

# Iterate over each folder in folder_palabras
for folder in folder_palabras:
  # Get all video files (ending with '.mp4') in the current folder
  videos = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.mp4')]
  # Append the list of video files to the video_files list
  video_files.append(videos)

In [ ]:
video_names = []

# Iterate over each folder in folder_palabras
for folder in folder_palabras:
    # Get the names of video files (without extension) in the current folder
    videos = [os.path.splitext(f)[0] for f in os.listdir(folder) if f.endswith('.mp4')]
    # Append the list of video names to the video_names list
    video_names.append(videos)

In [ ]:
def get_frames(video):
  #Extract frames from a video file
  video = cv2.VideoCapture(video)
  frames=[]

  while video.isOpened():
      ret, frame = video.read()
      
      if not ret:
          break 
      
      frames.append(frame)
  
  video.release()
  
  return frames,len(frames)

In [ ]:
def extraer_puntos_frame(frame, output_image, prev_points):
    # Extract hand landmarks from a frame and draw them on the output image
    height, width, _ = frame.shape
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands

    # Initialize mediapipe Hands module
    with mp.solutions.hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) as hands:
        # Process the frame
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Check if hand landmarks were detected
        if not results.multi_hand_landmarks:
            return output_image, prev_points

        # Iterate over detected hand landmarks
        for hand_handedness, hand_landmarks in zip(results.multi_handedness, results.multi_hand_landmarks):
            hand_label = hand_handedness.classification[0].label  # "Left" o "Right"
            finger_landmarks = [
                hand_landmarks.landmark[mp.solutions.hands.HandLandmark.WRIST],  # Añade la muñeca aquí
                hand_landmarks.landmark[mp.solutions.hands.HandLandmark.THUMB_TIP],
                hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP],
                hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP],
                hand_landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_TIP],
                hand_landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_TIP]
            ]

            # Select colors based on the hand label
            if hand_label == 'Left':
                colors = [(0, 165, 255), (255, 0, 0), (255, 165, 0), (255, 255, 0), (0, 255, 0), (0, 0, 255)]  # Añade un color para la muñeca
            else:
                colors = [(127, 127, 127), (255, 0, 255), (0, 255, 255), (127, 255, 212), (75, 0, 130), (238, 130, 238)]  # Añade un color para la muñeca
            
            # Iterate over finger landmarks
            for idx, landmark in enumerate(finger_landmarks):
                color = colors[idx]
                x, y = int(landmark.x * width), int(landmark.y * height)
                cv2.circle(output_image, (x, y), 5, color, -1)

                # Store coordinates of the point in the corresponding list
                prev_points[hand_label][idx].append((x, y))

                # Draw a line between each consecutive pair of points in the list
                for i in range(1, len(prev_points[hand_label][idx])):
                    cv2.line(output_image, prev_points[hand_label][idx][i-1], prev_points[hand_label][idx][i], color, 2)

    return output_image, prev_points


In [ ]:
def dibujar_puntos_caracteristicos(frame,output_image):
    #Draw characteristic feature points on the face in the frame.
    height, width, _ = frame.shape
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.3)
    results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Check if face landmarks were detected
    if not results.multi_face_landmarks:
        return output_image,0
    for face_landmarks in results.multi_face_landmarks:
        # Indices of landmark points for eyes, nose, mouth, and ears
        landmark_indices = {
            'left_eye': 33,      # Midpoint of the left eye
            'right_eye': 263,    # Midpoint of the right eye
            'nose_tip': 4,       # Tip of the nose
            'mouth_center': 0,   # Center of the mouth (midpoint between lips)
            'left_ear': 234,     # General area associated with the left ear
            'right_ear': 454     # General area associated with the right ear
        }

        # Iterate over landmark indices and draw feature points
        for feature, idx in landmark_indices.items():
            landmark = face_landmarks.landmark[idx]
            x, y = int(landmark.x * width), int(landmark.y * height)

            # Draw each feature point with a unique color and larger size
            cv2.circle(output_image, (x, y), 12, (170, 232, 238), -1)  
    return output_image,1


In [ ]:
def extraer_puntos_video(video):
    #Extract hand landmarks and facial feature points from a video
    frames, num_frames = get_frames(video)
    height, width, _ = frames[0].shape
    output_image = np.zeros((height, width, 3), dtype=np.uint8)
    
    # Dictionary to store previous hand landmarks for each hand label
    prev_points = {
    "Left":  [[] for _ in range(6)],  
    "Right": [[] for _ in range(6)],  
    }

    # Extract hand landmarks for each fram
    for frame in frames:
        output_image, prev_points = extraer_puntos_frame(frame, output_image, prev_points)
    
    # Draw facial feature points on the first frame
    for i in range (0,10):
      output_image,n_rostros=dibujar_puntos_caracteristicos(frames[0],output_image)
      print(n_rostros)
      if(n_rostros==1):
         break
    
    return output_image

In [ ]:
folder_imagenes = r"/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/"

# List files in the folder
!ls "{folder_imagenes}"

'01 Gracias'	     '05 Papa'	   '09 Abuelo'	 '13 Esposo'	'17 Puente'
'02 Muchas_gracias'  '06 Mama'	   '10 Tio'	 '14 Primo'	'18 Calle'
'03 Bienvenidos'     '07 Hermano'  '11 Sobrino'  '15 Casa'	'19 Avenida'
'04 Lo_espero'	     '08 Hijo'	   '12 Esposa'	 '16 Hospital'	'20 Plaza'


In [ ]:
path=folder_imagenes+folder_names[5]
i=1

# Iterate over each video in the video_files list for the specified folder
for video in video_files[5]:
  # Extract hand landmarks and facial feature points from the video
  img=extraer_puntos_video(video)

  # Resize the image to a desired size
  img = cv2.resize(img, (720, 480))

  # Define the path and file name for saving the image
  path_img=path+"/"+str(i)+".jpg"

  # Save the image to the specified path
  cv2.imwrite(path_img, img)

  # Print the path of the saved image
  print(path_img)

  # Increment the counter
  i+=1

1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/1.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/2.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/3.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/4.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/5.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/6.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/7.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/8.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/9.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/10.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/11.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/12.jpg
1
/content/drive/MyDrive/VISION_PROYECTO/Dataset_imagenes_v2/06 Mama/13.jpg
1
/content/drive/MyDr